In [3]:
%pip install openai
from pprint import pprint
import openai
import os

Note: you may need to restart the kernel to use updated packages.


In [4]:
secretKey = process.env.OPENAI_API_KEY
openai.api_key = secretKey


NameError: name 'process' is not defined